In [1]:
import warnings
warnings.filterwarnings("ignore")

import logging

logging.basicConfig(level=logging.ERROR)

In [2]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [3]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [5]:
files = sorted(glob('output-tamil/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

12954

In [6]:
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'
matches = re.findall(pattern_pair, '<|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan')
matches

[('0.00', ' kerajaan persekutuan', '1.46'),
 ('1.46', ' dan banyak masalah hubungan', '3.96')]

In [7]:
import string

punct = set(string.punctuation)

def remove_punct(s):
    return ''.join([c for c in s if c not in punct])

def remove_duplicate(string, n = 3):
    splitted = string.split()
    n = [splitted[i: i + n] for i in range(0, len(splitted), n)]
    already = set()
    dedup = []
    for n_ in n:
        original_n = ' '.join(n_)
        n_ = ' '.join(n_).lower()
        n_ = remove_punct(n_)
        if n_ not in already:
            dedup.append(original_n)
            already.add(n_)
    return ' '.join(dedup)

In [8]:
import math

def round_to_nearest_0_02(number):
    return round(number * 50) / 50

In [9]:
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
]

In [10]:
with open(files[0]) as fopen:
    d = json.load(fopen)
    
d[0]

{'predict_ta': [50258,
  50287,
  50360,
  50365,
  13535,
  4516,
  6980,
  5036,
  5511,
  12008,
  18169,
  4234,
  2133,
  4234,
  3923,
  36001,
  4742,
  2133,
  4742,
  3923,
  18198,
  19433,
  6467,
  8402,
  8491,
  4516,
  6993,
  2133,
  12008,
  5036,
  4742,
  3923,
  4234,
  5036,
  8012,
  2133,
  8012,
  3923,
  12776,
  6980,
  2133,
  16793,
  17167,
  2133,
  15094,
  2133,
  23656,
  6993,
  4234,
  4742,
  2133,
  4742,
  6467,
  4742,
  2133,
  4742,
  4516,
  6993,
  2133,
  23656,
  10078,
  2133,
  10078,
  12008,
  18169,
  4742,
  3923,
  6980,
  2133,
  12776,
  8708,
  8094,
  2133,
  8094,
  8402,
  16504,
  10478,
  6993,
  3923,
  6980,
  2133,
  16504,
  10478,
  6980,
  2133,
  35032,
  3923,
  4742,
  2133,
  4742,
  13535,
  10478,
  15297,
  2133,
  6421,
  4516,
  8491,
  4742,
  3923,
  12776,
  8708,
  16793,
  4516,
  8491,
  6980,
  2133,
  12894,
  5511,
  2133,
  8094,
  3923,
  13786,
  3923,
  8094,
  4516,
  8012,
  2133,
  8012,
  4516,


In [11]:
from huggingface_hub import hf_hub_download

f = hf_hub_download(
    repo_id="mesolitica/pseudolabel-tamil-large-v3-timestamp", 
    filename="translate-tamil-ms.json",
    repo_type = 'dataset'
)

translate-tamil-ms.json:   0%|          | 0.00/394M [00:00<?, ?B/s]

In [12]:
with open(f) as fopen:
    translation = json.load(fopen)

len(translation)

481437

In [13]:
len(files)

12954

In [14]:
import mp
import copy

minimum_score = 5

def loop(files):
    
    tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')
    
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = data[i]['filename']
            if not os.path.exists(audio_filename):
                continue
                
            y = audio.decode_example(audio.encode_example(audio_filename))['array']
            len_y = len(y) / sr
            if len_y > 30:
                continue
            rounded_num = f'<|{round_to_nearest_0_02(len_y):.2f}|>'
                    
            
            if data[i]['score_ta'] > minimum_score:
                
                a = tokenizer._decode_asr(
                [{'tokens': np.array([data[i]['predict_ta']])}], 
                return_timestamps = True, return_language = 'ta', 
                time_precision = 0.02)[1]['chunks']
            
                a = [a_['text'] for a_ in a]
                
                t = ' |'.join(a)
                if t not in translation:
                    continue
                    
                t_translated = translation[t]
                t_translated_splitted = t_translated.split('|')
                if len(t_translated_splitted) != len(a):
                    continue
                
                mapping = {}
                for k in range(len(a)):
                    s = t_translated_splitted[k].strip()
                    if a[k][0] == ' ':
                        s = ' ' + s
                    if a[k][-1] == ' ':
                        s = s + ' '
                    mapping[a[k]] = s
            
                a = np.array(data[i]['predict_ta'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        if t_ in mapping:
                            t_ = mapping[t_]
                        else:
                            t_ = ''
                    
                    splitted = t_.split()
                    if len(splitted):
                        ratio = (len(set(splitted)) / len(splitted))
                        if len(t_) > 100 and ratio < 0.5:
                            t_ = remove_duplicate(t_)
                    if len(t_) and t_[0] != ' ':
                        t_ = ' ' + t_
                            
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                    
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|ms|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
    return results

In [15]:
results = loop((files[:10], 0))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
 10%|█         | 1/10 [00:00<00:08,  1.09it/s]Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.
 20%|██        | 2/10 [00:01<00:04,  1.96it/s]Whisper did not predict an ending timestamp, which can happen 

In [16]:
len(results)

347

In [17]:
results[:1]

[{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Beliau berkata hari ini walaupun ciri keselamatan dalam sektor penerbangan bertaraf dunia, ia perlu dipertingkatkan lagi.<|8.20|><|endoftext|>',
  'audio_filename': '/home/ubuntu/newsonair_v5/tamil/NSD-Tamil-Tamil-0715-0725-201810118533/sent_13.wav'}]

In [18]:
[r for r in results if r['new_text'].count('<|') > 6]

[{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Jika pelajar yang berpindah ke institut pendidikan lain tidak mengembalikan yuran pengajian, kolej yang berkenaan dan giginya akan diambil kerana berjalan di helang.<|10.98|><|10.98|> Jabatan Sumber Manusia Pusat akan kekal.<|13.58|><|endoftext|>',
  'audio_filename': '/home/ubuntu/newsonair_v5/tamil/NSD-Tamil-Tamil-0715-0725-201810118533/sent_39.wav'},
 {'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Dia juga menunjukkan bahawa pendidikan pra-sekolah telah dilarang.<|6.00|><|6.00|> Apa yang anda dengar adalah laporan berita Akashavani.<|10.14|><|endoftext|>',
  'audio_filename': '/home/ubuntu/newsonair_v5/tamil/NSD-Tamil-Tamil-0715-0725-201810118533/sent_42.wav'},
 {'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Untuk menerbitkan Roll Pilihan Raya Iruthi yang diumumkan di Telangana,<|3.10|><|3.24|> Mahkamah Tinggi Hyderabad telah memberikan kebenaran kepada Suruhanjaya Pilihan Raya.<|7.14|

In [19]:
# tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')
# ori_tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

In [20]:
# tokenizer.tokenize(results[-1]['new_text'])

In [30]:
results = mp.multiprocessing(files, loop, cores = 50)

In [22]:
len(results)

442058

In [23]:
results[-2]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.00|> Setakat ini, 16.73 talian telah diuji untuk Covid-19.<|4.60|><|endoftext|>',
 'audio_filename': '/home/ubuntu/newsonair_v5/tamil/Regional-Tiruchirapalli-Tamil-1345-202158145135/sent_65.wav'}

In [24]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(results, test_size = 200)

In [25]:
with open('prepared-tamil-ms.jsonl', 'w') as fopen:
    for r in tqdm(train):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 441858/441858 [00:01<00:00, 266200.53it/s]


In [26]:
!ls -lh prepared-tamil-ms.jsonl

-rw-r--r-- 1 ubuntu ubuntu 131M Apr 27 10:32 prepared-tamil-ms.jsonl


In [27]:
from huggingface_hub import HfApi
api = HfApi()

In [28]:
api.upload_file(
    path_or_fileobj='prepared-tamil-ms.jsonl',
    path_in_repo='prepared-tamil-ms.jsonl',
    repo_id='mesolitica/pseudolabel-tamil-large-v3-timestamp',
    repo_type='dataset',
)

prepared-tamil-ms.jsonl:   0%|          | 0.00/137M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/pseudolabel-tamil-large-v3-timestamp/blob/main/prepared-tamil-ms.jsonl'

In [29]:
!wc -l prepared-tamil-ms.jsonl

441858 prepared-tamil-ms.jsonl


In [31]:
with open('test-tamil-dataset-ms.json', 'w') as fopen:
    json.dump(test, fopen)

In [32]:
api.upload_file(
    path_or_fileobj='test-tamil-dataset-ms.json',
    path_in_repo='ta-ms/test-tamil-dataset-ms.json',
    repo_id='mesolitica/speech-test-set',
    repo_type='dataset',
)

test-tamil-dataset-ms.json:   0%|          | 0.00/61.8k [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/speech-test-set/blob/main/ta-ms/test-tamil-dataset-ms.json'

In [36]:
!rm -rf ta-ms
!mkdir ta-ms

In [37]:
import shutil

for t in test:
    f = t['audio_filename'].replace('/', '-')
    shutil.copyfile(t['audio_filename'], os.path.join('ta-ms', f)) 
    
len(glob('ta-ms/*'))

200

In [42]:
# !sudo apt install zip -y

In [44]:
# !zip -r test-ta-ms-audio.zip ta-ms

In [45]:
api.upload_file(
    path_or_fileobj='test-ta-ms-audio.zip',
    path_in_repo='ta-ms/test-ta-ms-audio.zip',
    repo_id='mesolitica/speech-test-set',
    repo_type='dataset',
)

test-ta-ms-audio.zip:   0%|          | 0.00/39.7M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/speech-test-set/blob/main/ta-ms/test-ta-ms-audio.zip'